# Reshaping Data
## 데이터 재구성하기

## About the data
In this notebook, we will using daily temperature data from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will use the Global Historical Climatology Network - Daily (GHCND) dataset; see the documentation [here](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf).

This data was collected for New York City for October 2018, using the Boonton 1 station (GHCND:USC00280907). It contains:
- the daily minimum temperature (`TMIN`)
- the daily maximum temperature (`TMAX`)
- the daily temperature at time of observation (`TOBS`)

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Setup
시각화를 위해 데이터를 재구성한다.

In [6]:
import pandas as pd

long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head(10)

,datatype,date,temp_C,temp_F
0,TMAX,2018-10-01,21.1,69.98
1,TMIN,2018-10-01,8.9,48.02
2,TOBS,2018-10-01,13.9,57.02
3,TMAX,2018-10-02,23.9,75.02
4,TMIN,2018-10-02,13.9,57.02
5,TOBS,2018-10-02,17.2,62.96
6,TMAX,2018-10-03,25.0,77.00
7,TMIN,2018-10-03,15.6,60.08
8,TOBS,2018-10-03,16.1,60.98
9,TMAX,2018-10-04,22.8,73.04


* 사람들은 **주로 넓은 형태**로 데이터를 기록하고 제공하지만, (분석과 데이터베이스 설계에 주로 사용.)

* **특정 시각화**를 위해서는 **긴 형태**의 데이터가 필요하다. (긴 데이터 형태에서 각각의 열은 유한 의미가 있어야 하므로 나쁜 설계로 여겨짐.)

* **새로운 필드를 추가하거나 오래된 필드를 삭제**하는 경우, **긴 형태의 데이터**가 용이하다. -> 데이터베이스 사용자에게 고정된 스키마를 제공하면서 필요할 때마다 데이터를 업데이트할 수 있다. 

## Transposing
## 데이터 전치
**행과 열을 전치**시킨다. `T`속성 이용 

In [8]:
long_df.set_index('date').head(10).T #long_df 자체가 변환하진 않는다. 변환하려면 할당 필요.

date,2018-10-01,2018-10-01,2018-10-01,2018-10-02,2018-10-02,2018-10-02,2018-10-03,2018-10-03,2018-10-03,2018-10-04
datatype,TMAX,TMIN,TOBS,TMAX,TMIN,TOBS,TMAX,TMIN,TOBS,TMAX
temp_C,21.1,8.9,13.9,23.9,13.9,17.2,25.0,15.6,16.1,22.8
temp_F,69.98,48.02,57.02,75.02,57.02,62.96,77.0,60.08,60.98,73.04


## Pivoting
## 피보팅
**긴 형태**의 데이터를 대각선을 중심으로 회전시켜 **넓은 형태**의 데이터로 바꾸는 것을 **피봇**이라고 한다??

**전치(T)와 같은 효과**

### `pivot()`
먼저 **index** 인수로 date를 설정해주고

넓은 형태에서 **열의 이름**이 될 값을 가진 열: **columns** 인수로 지정.

피보팅을 할 때에는 열의 **값**이 될 열: **values** 인수로 지정

In [10]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values='temp_C'
)
pivoted_df.head(10) #long_df와의 바뀐 점은 date가 인덱스가 되고, datatype이라는 컬럼의 값들이 새로운 컬럼이 됨.

datatype,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9
2018-10-06,20.0,13.3,16.1
2018-10-07,20.0,16.1,20.0
2018-10-08,26.7,17.8,17.8
2018-10-09,18.9,17.2,17.8


넓은 형태의 데이터가 되었고, 이에 대한 요약통계는

In [11]:
pivoted_df.describe()

datatype,TMAX,TMIN,TOBS
count,31.000000,31.000000,31.000000
mean,16.829032,7.561290,10.022581
std,5.714962,6.513252,6.596550
min,7.800000,-1.100000,-1.100000
25%,12.750000,2.500000,5.550000
50%,16.100000,6.700000,8.300000
75%,21.950000,13.600000,16.100000
max,26.700000,17.800000,21.700000


We can also provide multiple values to pivot on, which will result in a hierarchical index:
피보팅을 하면서 다중 값을 넣어줄 수도 있다. (계층구조로 나타남.)

In [12]:
pivoted_df = long_df.pivot(
    index='date', columns='datatype', values=['temp_C', 'temp_F']
)

In [13]:
pivoted_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

With the hierarchical index, if we want to select `TMIN` in Fahrenheit, we will first need to select `temp_F` and then `TMIN`:

위와 같이 계층적 인덱스에서 **TMIN 값**을 뽑으려면, **먼저 temp_F인지** temp_C값인지 지정해주어야 한다.

In [14]:
pivoted_df['temp_F']['TMIN'].head()

date
2018-10-01    48.02
2018-10-02    57.02
2018-10-03    60.08
2018-10-04    53.06
2018-10-05    53.06
Name: TMIN, dtype: float64

### `unstack()`
판다스에서 제공하는 데이터 재구성 메서드 중 하나로 다중 인덱스를 가진 데이터 프레임 구조를 변경하는데 사용된다.
**다중인덱스**를 푸는 것이 **unstack**으로 보면 됨.
다단계 인덱스를 한 단계의 열로 내리는 방법.

주로
* 다중인덱스로 구성된 데이터 프레임을 "피벗"하는데에 사용.
* 스택(stack)된 형태에서 언스택(unstack)된 형태로 변환하는데에 사용.

In [15]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', 'datatype'])

* `set_index()`를 사용해서 여러 개의 인덱스를 설정할 수 있다. `MultiIndex` 타입이다.
* 순서가 중요하다.

데이터프레임에 두 개의 인덱스 섹션이 나뉘었다.

In [17]:
multi_index_df.head(10)

temp_C  temp_F
date       datatype                
2018-10-01 TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96
2018-10-03 TMAX        25.0   77.00
           TMIN        15.6   60.08
           TOBS        16.1   60.98
2018-10-04 TMAX        22.8   73.04

* `MultiIndex`를 사용하면 `pivot()`을 사용할 필요가 없다.
* `unstack()` 메서드를 이용해서 위의 multi_index_df를 여러개의 인덱스를 쌓지 않는? 것 같다.

In [18]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

temp_C             temp_F              
datatype     TMAX  TMIN  TOBS   TMAX   TMIN   TOBS
date                                              
2018-10-01   21.1   8.9  13.9  69.98  48.02  57.02
2018-10-02   23.9  13.9  17.2  75.02  57.02  62.96
2018-10-03   25.0  15.6  16.1  77.00  60.08  60.98
2018-10-04   22.8  11.7  11.7  73.04  53.06  53.06
2018-10-05   23.3  11.7  18.9  73.94  53.06  66.02

* `unstack()` 메서드는 데이터
를 재구성할 때 발생할 수 있는 NaN 값들을 채우기 위한 `fill_value` 매개변수도 제공한다.
* 아래 데이터를 보면, 평균온도가 2018년 10월 1일에만 존재한다.

In [26]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_30388/3903132469.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extra_data = long_df.append([{
/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_30388/3903132469.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  extra_data = long_df.append([{


temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96

In [27]:
#append 함수말고 pd.concat 함수로 추가하라고 함.
extra_data2 = pd.concat([long_df, pd.DataFrame([{
    'datatype': 'TAVG',
    'date': '2018-10-01',
    'temp_C': 10,
    'temp_F': 50
}])], ignore_index=True).set_index(['date', 'datatype']).sort_index()

extra_data2['2018-10-01':'2018-10-02']

/var/folders/rc/3jzqjhp56_v2qxb49fkb1x880000gn/T/ipykernel_30388/36527116.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  extra_data2 = pd.concat([long_df, pd.DataFrame([{


temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96

In [30]:
# 문자열이 포함된 데이터로부터 datetime64[ns] 타입을 추론하는 것이 미래 버전에서 지원되지 않을 것임을 알려주는 것
# -> 데이터프레임에서 날짜와 관련된 열을 datetime64[ns] 타입으로 명시적으로 변환해야함.
new_data = pd.DataFrame([{
    'datatype': 'TAVG',
    'date': '2018-10-01',
    'temp_C': 10,
    'temp_F': 50
}])

# 'date' 열을 datetime64[ns] 타입으로 변환
new_data['date'] = pd.to_datetime(new_data['date'])

# 기존 데이터프레임과 새로운 데이터프레임을 연결하고 인덱스를 재설정
extra_data3 = pd.concat([long_df, new_data], ignore_index=True).set_index(['date', 'datatype']).sort_index()
extra_data3.head(10)

temp_C  temp_F
date       datatype                
2018-10-01 TAVG        10.0   50.00
           TMAX        21.1   69.98
           TMIN         8.9   48.02
           TOBS        13.9   57.02
2018-10-02 TMAX        23.9   75.02
           TMIN        13.9   57.02
           TOBS        17.2   62.96
2018-10-03 TMAX        25.0   77.00
           TMIN        15.6   60.08
           TOBS        16.1   60.98

위와 같은 상태에서 `unstack()` 메서드를 사용하면 TAVG 값을 채우기 위해 `NaN`값이 채워진다.

In [32]:
extra_data3.unstack().head()

temp_C                   temp_F                     
datatype     TAVG  TMAX  TMIN  TOBS   TAVG   TMAX   TMIN   TOBS
date                                                           
2018-10-01   10.0  21.1   8.9  13.9   50.0  69.98  48.02  57.02
2018-10-02    NaN  23.9  13.9  17.2    NaN  75.02  57.02  62.96
2018-10-03    NaN  25.0  15.6  16.1    NaN  77.00  60.08  60.98
2018-10-04    NaN  22.8  11.7  11.7    NaN  73.04  53.06  53.06
2018-10-05    NaN  23.3  11.7  18.9    NaN  73.94  53.06  66.02

unstack() 메서드로 다중 인덱스를 풀 때 생기는 `NaN`값들을 -40으로 일괄적으로 채워줄 수 있다.

In [34]:
extra_data.unstack(fill_value=-40).head()

temp_C                   temp_F                     
datatype     TAVG  TMAX  TMIN  TOBS   TAVG   TMAX   TMIN   TOBS
date                                                           
2018-10-01   10.0  21.1   8.9  13.9   50.0  69.98  48.02  57.02
2018-10-02  -40.0  23.9  13.9  17.2  -40.0  75.02  57.02  62.96
2018-10-03  -40.0  25.0  15.6  16.1  -40.0  77.00  60.08  60.98
2018-10-04  -40.0  22.8  11.7  11.7  -40.0  73.04  53.06  53.06
2018-10-05  -40.0  23.3  11.7  18.9  -40.0  73.94  53.06  66.02

## Melting
## 멜팅
* 넓은 데이터 형태에서 긴 데이터 형태로 변환할 때에 **멜팅**을 사용한다. (**넓 -> 긴**)

### Setup

In [36]:
wide_df = pd.read_csv('data/wide_data.csv') #datetype의 값들이 column으로
wide_df.head()

,date,TMAX,TMIN,TOBS
0,2018-10-01,21.1,8.9,13.9
1,2018-10-02,23.9,13.9,17.2
2,2018-10-03,25.0,15.6,16.1
3,2018-10-04,22.8,11.7,11.7
4,2018-10-05,23.3,11.7,18.9


### `melt()`
**멜팅**은 피봇 작업을 되돌리는 것. 

넓은 포맷에서 긴 포맷으로 변환하기 위해서는 우리는 `melt()` 메서드를 사용한다. 
- `id_vars`: 하나의 row를 구별하는 유일한 값 (여기서는 `date` 컬럼)
- `value_vars`: 값으로 들어가게 되는 컬럼들(여기서는 `TMAX`, `TMIN`, and `TOBS` 컬럼들)

선택적으로 우리는 아래와 같은 매개변수를 넣을 수 있다. (**새롭게 컬럼이름 지정**)
- `value_name`: 값이 들어가는 컬럼 지정.
- `var_name`: value_vars를 묶는 하나의 컬럼 이름.

In [42]:
melted_df = wide_df.melt(
    id_vars='date', #식별자 역할
    value_vars=['TMAX', 'TMIN', 'TOBS'], #TMAX, TMIN, TOBS 열들의 값을 온도의 단일 열(value_vars)로 변환하고,
    var_name='measurement', #measurement라는 열의 값으로 가용한다.
    value_name='temp_C' #마지막으로 값의 열을 temp_C로 지정한다.    
)
melted_df.head(100)

,date,measurement,temp_C
0,2018-10-01,TMAX,21.1
1,2018-10-02,TMAX,23.9
2,2018-10-03,TMAX,25.0
3,2018-10-04,TMAX,22.8
4,2018-10-05,TMAX,23.3
...,...,...,...
88,2018-10-27,TOBS,6.1
89,2018-10-28,TOBS,7.2
90,2018-10-29,TOBS,8.3
91,2018-10-30,TOBS,5.0


### `stack()`

`unstack()` 메서드를 이용해 데이터를 피보팅하는 방법이 있었듯이,

`stack()` 메서드를 이용해 데이터를 멜팅하는 방법도 있다.

In [43]:
wide_df.set_index('date', inplace=True)
wide_df.head()

,TMAX,TMIN,TOBS
date,,,
2018-10-01,21.1,8.9,13.9
2018-10-02,23.9,13.9,17.2
2018-10-03,25.0,15.6,16.1
2018-10-04,22.8,11.7,11.7
2018-10-05,23.3,11.7,18.9


`stack()` 메서드를 사용해 인덱스를 두 레벨로 생성할 수 있다. 

이 때 컬럼의 이름들이 두번째 레벨의 인덱스로 들어가게 된다.(`TMAX`, `TMIN`, `TOBS`) 

In [44]:
stacked_series = wide_df.stack()
stacked_series.head() #해당 결과는 series 객체

date            
2018-10-01  TMAX    21.1
            TMIN     8.9
            TOBS    13.9
2018-10-02  TMAX    23.9
            TMIN    13.9
dtype: float64

`to_frame()` 메서드를 이용해 데이터의 열 이름을 지정하여 DataFrame 객체로 만들 수 있다.

In [45]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

values
date                   
2018-10-01 TMAX    21.1
           TMIN     8.9
           TOBS    13.9
2018-10-02 TMAX    23.9
           TMIN    13.9

현재 `MultiIndex` 갖고 있지만, 인덱스의 이름으로 **date와 datatype**이 들어가야하지만, 
datatype은 **None**으로 뜬다.

In [46]:
stacked_df.head().index

MultiIndex([('2018-10-01', 'TMAX'),
            ('2018-10-01', 'TMIN'),
            ('2018-10-01', 'TOBS'),
            ('2018-10-02', 'TMAX'),
            ('2018-10-02', 'TMIN')],
           names=['date', None])

In [47]:
stacked_df.index.names

FrozenList(['date', None])

`set_names()`메서드로 이를 해결한다.

In [48]:
stacked_df.index.set_names(['date', 'datatype'], inplace=True)
stacked_df.index.names

FrozenList(['date', 'datatype'])

<hr>
<div>
    <a href="./3-cleaning_data.ipynb">
        <button>&#8592; Previous Notebook</button>
    </a>
    <a href="./5-handling_data_issues.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>